In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
df=pd.read_csv("E:\\archive\\spam.csv",encoding='latin1')

In [4]:
df

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


# Data preprocessing
## Remove the unwanted columns

In [5]:
df.drop(columns=["Unnamed: 2","Unnamed: 3","Unnamed: 4"], axis = 1, inplace = True)
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


##  Rename the columns appropriately

In [6]:
df.rename(columns={"v1": "label","v2": "messages"},inplace=True)
df.head()

,label,messages
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [7]:
df=df[["messages","label"]]
df.head()

,messages,label
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham


In [8]:
df.isna().sum()

messages    0
label       0
dtype: int64

In [9]:
df[df.duplicated()==True]

,messages,label
102,As per your request 'Melle Melle (Oru Minnamin...,ham
153,As per your request 'Melle Melle (Oru Minnamin...,ham
206,"As I entered my cabin my PA said, '' Happy B'd...",ham
222,"Sorry, I'll call later",ham
325,No calls..messages..missed calls,ham
...,...,...
5524,You are awarded a SiPix Digital Camera! call 0...,spam
5535,"I know you are thinkin malaria. But relax, chi...",ham
5539,Just sleeping..and surfing,ham
5553,Hahaha..use your brain dear,ham


In [10]:
df.duplicated().sum()

403

## Remove the duplicates values 

In [14]:
df.drop_duplicates(inplace=True)

C:\Users\cheta\AppData\Local\Temp\ipykernel_14200\3006716147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [12]:
df.duplicated().sum()

0

## Remove the categorical values into numerical values

In [16]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df["label"] = encoder.fit_transform(df["label"])
df.head()

C:\Users\cheta\AppData\Local\Temp\ipykernel_14200\3021016900.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["label"] = encoder.fit_transform(df["label"])


,messages,label
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0


In [17]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
x=vectorizer.fit_transform(df["messages"])
x.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [18]:
y = df["label"]

In [19]:
x.shape

(5169, 8672)

In [20]:
y.shape

(5169,)

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [22]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=100)

In [23]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver="liblinear")
model.fit(x_train,y_train)
predictions = model.predict(x_test)

In [24]:
accuracy_score(y_test, predictions)

0.9806576402321083

In [25]:
def is_message_spam(message):
    vectorized_message = vectorizer.transform([message])
    if model.predict(vectorized_message) == 1:
        print("spam!!!")
    else:
        print("Not spam")


In [31]:
is_message_spam("congratulations! you've won a luxury vacation for two.click the link to claim:https//fakenews.com")

spam!!!


In [32]:
is_message_spam("hello. how are you")

Not spam


In [34]:
is_message_spam("Urgent: your bank account gets locked. reply with your login details to unlocked")

spam!!!
